# Übung: Bin packing

Paketlieferdienst *Superfix* will die Lieferungen, die für einen Tag anstehen, auf Lieferwagen verteilen. *Superfix* benutzt hierfür identische Lieferwagen, die einen Lagervolumen von $3 m^3$ haben. Der Einfachheit halber nehmen wir an, dass alle Pakete lückenlos in einen Lieferwagen eingeräumt werden können. Das Volumen der Pakete ist also die einzige Restriktion.

Ziel des Lieferdienstes ist nun alle Pakete eines Tages so auf Lieferwagen aufzuteilen, sodass so wenig Lieferwagen wie möglich benötigt werden.

# Methoden zur Veranschaulichung

Hier ist ein wenig Visualisierungscode: Mit der Methode <code>show_vehicles(packages,assignment)</code> können Sie sich die Verteilung der Pakete auf die Lieferwagen anzeigen lassen. Dabei beschreibt der Parameter <code>packages</code> die Instanz, während <code>assignment</code> die Verteilung darstellt.

Führen Sie die Zelle einfach aus. Wir werden den Code nicht weiter diskutieren.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
import random

def show_vehicles(packages,assignment):
    num_trucks = len(assignment)
    if num_trucks <= 0:
        return
    
    # Declaring a figure "ax" 
    figure,ax = plt.subplots(figsize=(10,1.5*num_trucks)) 
    
    # Setting axis limits 
    ax.set_xlim(-0.3,  VOLUME*1.05) 
    ax.set_ylim(-1,3*num_trucks)
    # Setting labels for x-axis and y-axis 
    ax.set_xlabel('Füllmenge') 

    # Setting ticks on y-axis 
    ax.set_yticks(range(1,3*num_trucks+1,3)) 
    # Labelling tickes of y-axis 
    ax.set_yticklabels(range(1,num_trucks+1)) 
    for i in range(num_trucks):
        current = 0
        ax.add_artist(Ellipse(xy=(-0.1,3*i),width=0.7*1.5/10,height=0.7,color='black'))
        ax.add_artist(Ellipse(xy=(VOLUME*0.8,3*i),width=0.7*1.5/10,height=0.7,color='black')) 
        ax.broken_barh([(-0.3, 0.28)], (3*i, 1.2),color='black',edgecolor='black')
        ax.broken_barh([(-0.2, 0.18)], (3*i, 2),color='black',edgecolor='black')
        ax.broken_barh([(-0.2, 0.2+VOLUME)], (3*i, 0.01),color='black',edgecolor='black')
        ax.broken_barh([(-0.18, 0.14)], (3*i+1.3, 0.4),color='white',edgecolor='white') 
        for package_id in assignment[i]:
            t=packages[package_id][1]
            # Declaring a bar 
            ax.broken_barh([(current, t)], (3*i, 2),edgecolor='black') 
            ax.text(x=current+t/2 , 
                        y=3*i+1,
                        s=package_id, 
                        ha='center', 
                        va='center',
                        color='white',
                       )
            current += t
    ax.set_title('Lieferwagen')
    plt.show()

## Das Problem

Das Problem ist durch eine Liste von Pakete charakterisiert. Diese Liste, <code>packages</code>, besteht aus Paaren <code>(27,1.328)</code>, wobei im ersten Eintrag die Paketnummer und im zweiten Eintrag das Paketvolumen steht. Weiterhin gibt's eine globale Variable <code>VOLUME</code>, in der das Fassungsvermögen der Lieferwagen festgehalten ist. Die Methode <code>create_instance</code> erzeugt eine Instanz.

In [ ]:
VOLUME = 3 # Volumen eines Lieferwagens

def create_instance(number=30,seed=-1):
    if seed>0:
        random.seed(seed)
    return [(t,0.5*random.random()+0.02+((0.75*t/number) > random.random())*1) for t in range(number)]

packages = create_instance(number=11)
packages

Wir sehen, die Paketnummern sind, ein wenig langweilig, einfach fortlaufend. Die Paketgrößen schwanken zwischen recht klein bis über $1m^3$.

Wir testen auch den Visualisierungscode. Dazu brauchen wir eine Zuteilung der Pakete auf die Lieferwagen. Wie kodieren wir die? Einfach! Als Liste von Listen. Jede Teilliste steht für einen Lieferwagen und enthält die Paketnummern der Pakete, die in den jeweiligen Lieferwagen geladen werden. 

In [ ]:
assignment=[[0,1,9,10],[3,5,4],[2,6,7,8]]
show_vehicles(packages,assignment)

### Aufgabe: Greedy-Algorithmus
Implementieren Sie einen Greedy-Algorithmus! Der Algorithmus soll die Pakete der Reihe nach abarbeiten. Falls das Paket in einen bereits benutzen Lieferwagen passt, soll es einfach in den Lieferwagen geladen werden. Ansonsten wird das Paket in einen neuen Lieferwagen gelegt.

In [ ]:
def greedy(packages):
    ### Ihr Code hier ###

    ### Ende Ihres Codes ###

packages=create_instance(seed=42)
trucks = greedy(packages)
show_vehicles(packages,trucks)
print("Es werden {} Lieferwagen benötigt".format(len(trucks)))

## Sortieren

Oft kann der Greedy-Algorithmus durch eine gewissen Vorverarbeitung verbessert werden. Hier bietet es sich an, die Pakete vor Anwendung des Greedy-Algorithmus' zu sortieren, und zwar so, dass die großen Pakete als erstes verladen werden. Python bieten dazu die Methode <code>sorted</code>. 

Als kleine Schwierigkeit besteht unsere Instanzliste <code>packages</code> aus Paaren <code>(PaketId,PaketVol)</code>. Wenn wir <code>sorted</code> direkt auf die Liste anwenden, dann werden die Pakete nach der Paketnummer sortiert -- was wenig hilfreich ist. Um die Pakete nach dem Volumen zu sortieren, müssen wir über den Parameter <code>key</code> spezifieren, wie genau sortiert werden soll. Mit der Hilfsmethode <code>itemgetter</code> können wir einstellen, dass jeweils das zweite Element jedes Paares, eben gerade das Paketvolumen, maßgeblich für die Sortierung ist. Hier ist ein Beispiel:

In [ ]:
some_packages=[(42,12.5),(1010,3.142),(12,100.6),(27,1.234)]

from operator import itemgetter
sorted(some_packages,key=itemgetter(1)) # warum "(1)"? Weil wir das zweite Element wollen -- also Element 1, da die Zählung bei 0 startet

Wir wollen, dass die großen Pakete zuerst verladen werden. Dazu müssen wir von groß nach klein sortieren. Der Parameter <code>reverse=True</code> hilft dabei.

### Aufgabe: Greedy mit Sortierung

Implementieren Sie einen greedy-Algorithmus mit Sortierung. Dh, der Greedy-Algorithmus soll erst die großen Pakete auf die Lieferwagen verteilen, dann die mittelgroßen und am Ende die kleinen. 

Hinweis: Wenn Sie's geschickt anstellen, können Sie einfach den Greedy-Algorithmus von oben wieder verwenden.

In [ ]:
def sorted_greedy(packages):
    ### Ihr Code hier ###

    ### Ende Ihres Codes ###

packages=create_instance(seed=42)
trucks = sorted_greedy(packages)
show_vehicles(packages,trucks)
print("Es werden {} Lieferwagen benötigt".format(len(trucks)))

### Aufgabe: Systematischer Vergleich

Wir wollen sehen, ob Greedy mit Sortierung generell besser ist als Greedy ohne Sortierung (zumindest mit den Zufallsinstanzen, die wir hier erzeugen). Erzeugen Sie 10000 Zufallsinstanzen per Aufruf <code>create_instance()</code> und vergleichen Sie <code>greedy</code> mit <code>sorted_greedy</code>. Erfassen Sie in der Variable <code>sorted_better</code> wie oft <code>sorted_greedy</code> weniger Lieferwagen braucht, in der Variable <code>no_sort_better</code> wie oft <code>greedy</code> besser ist und in <code>ties</code> wie oft beide Algorithmen die gleiche Anzahl an Lieferwagen benötigen. Berechnen Sie weiterhin die durchschnittlichen Anzahlen der benötigten Lieferwagen und speichern Sie die in den Variablen <code>avg_sorted</code> und <code>avg_no_sort</code>.

In [ ]:
### Ihr Code hier ###

### Ende Ihres Codes ###

print("Mit Sortierung besser als ohne: {}".format(sorted_better))
print("Ohne Sortierung besser als mit: {}".format(no_sort_better))
print("Gleich gut:                     {}".format(ties))
print("")
print("Durchschnitt Lieferwägen mit Sortierung:  {:.1f}".format(avg_sorted))
print("Durchschnitt Lieferwägen ohne Sortierung: {:.1f}".format(avg_no_sort))